In [1]:
import random
import time

def main():
    # 역전 원소 개수 세기
    print('# 역전 원소 개수 세기')

    # 역전 알고리즘을 위한 무작위 리스트 생성.
    lst = [i for i in range(100)] * 50
    lst.sort(key = lambda e: random.random() < 0.5)

    start = time.time()
    brute_solution = inverse_count(lst)
    end = time.time()
    brute_time = end - start

    start = time.time()
    divide_and_conquer_solution = msort_inverse_count(lst)
    end = time.time()
    divide_and_conquer_time = end - start

    # print(f'Initla List: {lst}')
    print(f'Number of Elements: {len(lst)}')
    print(f'Brute Force: {brute_solution} ({brute_time} sec)')
    print(f'Divide and Conquer : {divide_and_conquer_solution} ({divide_and_conquer_time} sec)')

#### **# 마스터 정리를 이용한 시간 복잡도 계산**  

##### # 1.  
$$T(n) = 4\ T(n/2) + n$$
$$a = 4,\ b = 2,\ c = 1,\ O(f(n)) = O(n) → d = 1$$  
$$4 > 2^1 → T(n) = O(n^{\log_2 4}) = O(n^2)$$  

##### # 2.  
$$T(n) = 8\ T(n/4) + n^2$$  
$$a = 8,\ b = 4,\ c = 1,\ O(f(n)) = O(n^2) → d = 2$$  
$$8 < 4^2 → T(n) = O(n^2)$$  

##### # 3.  
$$T(n) = T(n/2) + n^3$$  
$$a = 1,\ b = 2,\ c = 1,\ O(f(n)) = O(n^3) → d = 3$$  
$$1 < 2^3 → T(n) = O(n^3)$$

In [2]:
# 브루트 포싱 역전 카운팅 알고리즘.
def inverse_count(lst):
    count = 0

    # 모든 가능한 조합에 대해서 수행.
    for i in range(len(lst)):
        for j in range(i + 1, len(lst)):
            # 두 원소가 역전된 관계이면 count++
            if i != j and lst[i] > lst[j]:
                count += 1
    return count

#### **# 브루트 포싱 알고리즘을 통해 구현한 역전 카운팅 알고리즘의 간단한 시간 복잡도 분석**  

브루트 포싱 기법을 이용하여 가능한 모든 원소의 조합에 대해 역전 여부를 조사하므로 시간 복잡도는 다음과 같이 나타낼 수 있다.  
$$O({n \choose 2})\quad(단,\ 괄호\ 안의\ n과\ 2는\ 조합을\ 의미)$$  

조합의 정의를 이용하면 위의 복잡도는 아래와 같다.  
$$O({n \choose 2}) = O(\frac{n(n-1)}{2}) = O(n^2)$$

In [3]:
# 분할정복 역전 카운팅 알고리즘을 설계하기 위한 분할정복 정렬 알고리즘 설계.

# 병합정렬의 인수 단순화를 위한 함수.
def msort(lst):
    __msort(lst, 0, len(lst) - 1)

# 병합정렬 주요 로직.
def __msort(lst, left, right):
    # 전달된 서브 리스트의 범위가 유효할 때 수행.
    if left < right:
        middle = (left + right) // 2                # 리스트의 중간 인덱스.
        __msort(lst, left, middle)                  # 좌측 서브 리스트에 대해 재귀적으로 수행.
        __msort(lst, middle + 1, right)             # 우측 서브 리스트에 대해 재귀적으로 수행.
        __msort_merge(lst, left, right, middle)     # 좌측 서브 리스트와 우측 서브 리스트 병합.

def __msort_merge(lst, left, right, middle):
    # 사용되는 인덱스 포인터 변수 설정.
    l = left            # 좌측 서브 리스트
    r = middle + 1      # 우측 서브 리스트

    # 정렬된 배열을 저장할 보조 리스트.
    sorted = []

    # 죄측 인덱스와 우측 인덱스 모두 유효할 때 수행.
    while l <= middle and r <= right:
        # 최측 인덱스가 적절한 원소를 갖고 있으면 이를 보조 리스트에 복사.
        if lst[l] < lst[r]:
            sorted.append(lst[l])
            l += 1

        # 우측 인덱스가 적절한 원소를 갖고 있으면 이를 보조 리스트에 복사.
        else:
            sorted.append(lst[r])
            r += 1

    # 남은 서브 리스트에 잔존하는 원소 복사.
    if (l > middle):
        while (r <= right):
            sorted.append(lst[r])
            r += 1
    else:
        while (l <= middle):
            sorted.append(lst[l])
            l += 1

    # 정렬된 보조 리스트를 메인 리스트에 반영.
    lst[left:right + 1] = sorted

In [4]:
# 분할정복 정렬 알고리즘을 이용한 분할정복 역전 카운팅 알고리즘.
def msort_inverse_count(lst):
    return __msort_inverse_count(lst.copy(), 0, len(lst) - 1)    # 역전된 원소의 개수를 반환하므로 return 사용.

def __msort_inverse_count(lst, left, right):
    if left < right:
        middle = (left + right) // 2
        left_count = __msort_inverse_count(lst, left, middle)
        right_count = __msort_inverse_count(lst, middle + 1, right)
        merge_count = __msort_inverse_count_merge(lst, left, right, middle)
        return left_count + right_count + merge_count            # 서브 리스트의 역전 원소의 개수까지 취합하여 반환.

    # 범위가 올바르지 않으면 역전된 원소를 정의할 수 없으므로 0 반환.
    return 0

def __msort_inverse_count_merge(lst, left, right, middle):
    l = left
    r = middle + 1

    count = 0

    sorted = []
    while l <= middle and r <= right:
        # 정렬된 상태라면 죄측 서브 리스트의 현재 원소가 우특 서브 리스트의 현재 원소보다 작아야 함.
        # 따라서 우측 서브 리스트 원소의 크기가 작으면 역전 원소의 개수 증가.
        if lst[l] <= lst[r]:
            sorted.append(lst[l])
            l += 1
        else:
            sorted.append(lst[r])
            r += 1

            # 양 서브 리스트는 정렬된 상태이므로 좌측 서브 리스트의 모든 원소가 우측 서브 리스트의 현재 원소보다 크다는 의미.
            # 우측 서브 리스트의 현재 원소는 좌측 서브 리스트의 모든 원소에 대해 역전됨.
            count += middle - l + 1 

    if (l > middle):
        while (r <= right):
            sorted.append(lst[r])
            r += 1
    else:
        while (l <= middle):
            sorted.append(lst[l])
            l += 1

    lst[left:right + 1] = sorted
    return count

#### **# 분할정복 알고리즘을 통해 구현한 역전 카운팅 알고리즘의 간단한 시간 복잡도 분석**  

##### **# 이진탐색 알고리즘의 시간 복잡도**  

위에서 구현한 알고리즘의 시간 복잡도를 구하려면 먼저 이진탐색의 시간 복잡도를 먼저 알아야 한다.  
일반적으로 이진탐색 알고리즘의 시간 복잡도는 다음과 같이 알려져 있다.  
$$O(\log_2 n)$$  
이는 이진탐색을 수행할 때 배열을 나누는 횟수와 관련이 있다.  

임의의 길이 n을 같는 배열을 생각해보자.  
이진탐색을 수행하면 각 반복에서 배열의 길이는 절반이 된다.  

한편 운이 나빠 찾고자 하는 원소가 가장 마지막에 나타난다고 가정하면, 그때의 배열의 길이는 1이 될 것이다.  
이를 수식으로 표현하면 다음과 같다.  
$$n * (1/2)^t = 1\quad(단,\ t는\ 이진탐색에서\ 각\ 단계를\ 수행한\ 횟수)$$  

위의 식에서 t를 구하면 이진탐색 알고리즘에서 배열을 나누는 횟수를 구할 수 있다.  
t를 구하기 위해 식을 정리하면 다음과 같다.  
$$n = 2^t$$  
$$t = \log_2 n$$  

이때 각 반복에서 수행하는 연산은 원소들끼리의 상수 번 비교이므로 이진탐색의 시간 복잡도는 다음과 같다.  
$$O(\log_2 2)$$  

##### **# 분할정복 정렬 알고리즘, 병합정렬의 시간 복잡도 분석**  

위에서 이진탐색의 시간 복잡도를 알아봤다. 이를 활용하면 병합정렬의 시간 복잡도는 쉽게 구할 수 있다.  

먼저 병합정렬은 배열을 계속해서 반으로 나누며 진행하므로 이때의 시간 복잡도는 이진탐색과 동일하다.  
$$O(\log_2 n)$$  

그런데 각각의 분할에서 각 원소들을 비교하고 복사하는 과정이 있다.  
배열의 길이가 n이라고 했을 때 각 원소들을 비교하고 복사하는 과정은 선형이므로 그 시간 복잡도는 다음과 같다.  
$$O(n)$$  

상술한 과정이 각각의 분할에서 이뤄지므로 최종적으로 병합정렬의 시간 복잡도는 다음과 같다.  
$$O(n\ \log_2 n)$$  

##### **# 분할정복 알고리즘을 이용해 구현한 역전 카운팅 알고리즘의 시간 복잡도 분석**

해당 알고리즘은 병합 정렬에서 count 변수를 할당하고 계산하고 반환하는 연산, 즉 상수의 시간 복잡도를 갖는 연산만 추가되었다.  
따라서 그 복잡도는 병합정렬과 같으므로 구하고자 하는 시간 복잡도는 다음과 같다.  
$$O(n\ \log_2 n)$$

In [5]:
if __name__ == '__main__':
    main()

# 역전 원소 개수 세기
Number of Elements: 5000
Brute Force: 6077818 (2.8639702796936035 sec)
Divide and Conquer : 6077818 (0.03731727600097656 sec)
